In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(logical_gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)


1 Physical GPUs, 1 Logical GPU


In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/v1-dataset-plant-village/Plant_leave_diseases_dataset_with_augmentation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 19083 images belonging to 19 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/v1-dataset-plant-village/Plant_leave_diseases_dataset_with_augmentation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 4766 images belonging to 19 classes.


In [8]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = models.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(19, activation='softmax'))  


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/10
596/596 [==============================] - 359s 571ms/step - loss: 3.1536 - accuracy: 0.0868 - val_loss: 2.9124 - val_accuracy: 0.0889
Epoch 2/10
596/596 [==============================] - 302s 506ms/step - loss: 2.9120 - accuracy: 0.0892 - val_loss: 2.9119 - val_accuracy: 0.0893
Epoch 3/10
596/596 [==============================] - 298s 499ms/step - loss: 2.9115 - accuracy: 0.0885 - val_loss: 2.9113 - val_accuracy: 0.0887
Epoch 4/10
596/596 [==============================] - 296s 496ms/step - loss: 2.9115 - accuracy: 0.0888 - val_loss: 2.9119 - val_accuracy: 0.0893
Epoch 5/10
596/596 [==============================] - 293s 491ms/step - loss: 2.9116 - accuracy: 0.0884 - val_loss: 2.9114 - val_accuracy: 0.0889
Epoch 6/10
596/596 [==============================] - 293s 491ms/step - loss: 2.9115 - accuracy: 0.0883 - val_loss: 2.9107 - val_accuracy: 0.0891
Epoch 7/10
596/596 [==============================] - 295s 495ms/step - loss: 2.9113 - accuracy: 0.0883 - val_loss: 2.9104 -

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

149/149 [==============================] - 61s 408ms/step - loss: 2.9101 - accuracy: 0.0892
Validation Accuracy: 8.92%


v2

In [12]:
from keras.applications.vgg16 import VGG16
import keras

In [13]:
strategy_1 = tf.distribute.MirroredStrategy()
with strategy_1.scope():
    base_model=VGG16(include_top=False,input_shape=(224,224,3))
    base_model.trainable=False
    model_1=keras.models.Sequential()
    model_1.add(base_model)
    model_1.add(layers.Flatten())
    model_1.add(layers.Dense(4096, activation='relu'))
    model_1.add(layers.Dense(4096, activation='relu'))
    model_1.add(layers.Dense(19, activation='softmax'))  


58889256/58889256 [==============================] - 0s 0us/step


In [14]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model_1.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

Epoch 1/10
596/596 [==============================] - 303s 498ms/step - loss: 0.9180 - accuracy: 0.7886 - val_loss: 0.3069 - val_accuracy: 0.8898
Epoch 2/10
596/596 [==============================] - 296s 497ms/step - loss: 0.2763 - accuracy: 0.9049 - val_loss: 0.2975 - val_accuracy: 0.8986
Epoch 3/10
596/596 [==============================] - 296s 497ms/step - loss: 0.2113 - accuracy: 0.9265 - val_loss: 0.3540 - val_accuracy: 0.8894
Epoch 4/10
596/596 [==============================] - 301s 504ms/step - loss: 0.1978 - accuracy: 0.9300 - val_loss: 0.2871 - val_accuracy: 0.9033
Epoch 5/10
596/596 [==============================] - 302s 507ms/step - loss: 0.1577 - accuracy: 0.9453 - val_loss: 0.2533 - val_accuracy: 0.9179
Epoch 6/10
596/596 [==============================] - 295s 495ms/step - loss: 0.1738 - accuracy: 0.9424 - val_loss: 0.1957 - val_accuracy: 0.9337
Epoch 7/10
596/596 [==============================] - 299s 501ms/step - loss: 0.1422 - accuracy: 0.9495 - val_loss: 0.2216 -

In [16]:
loss, accuracy = model_1.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

149/149 [==============================] - 60s 399ms/step - loss: 0.2385 - accuracy: 0.9232
Validation Accuracy: 92.32%


In [17]:
model.save('/kaggle/working/vgg16-v2.keras')

In [18]:
from IPython.display import FileLink
!zip -r file.zip /kaggle/working
FileLink(r'file.zip')

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/__notebook__.ipynb (deflated 97%)
  adding: kaggle/working/vgg16-v2.keras (deflated 42%)


/kaggle/working/file.zip

In [19]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [20]:
zip_dir()

/kaggle/working/directory.zip